In [1]:
# %% codecell
#!/usr/bin/python3
### get general libraries
import os
import sys

import pandas as pd

In [2]:
### get PDB credentials
cwd = os.getcwd()
# check where this is:
#cwd
sys.path.insert(1, cwd+"/Notebooks")
import setEnv
setEnv.SetEnv()

# import API and check authentication
import itkdb
myClient = itkdb.Client()
myClient.user.authenticate()
user=myClient.get('getUser', json={'userIdentity': myClient.user.identity})
print(user['firstName'])

Done
Kenneth


In [3]:
projCode="P"
compTypeCode="BARE_MODULE"
# new module
bareObj = myClient.get('generateComponentTypeDtoSample', json={'project':projCode, 'code': compTypeCode, 'requiredOnly' : False})

### add basic info.
bareObj['subproject']="PG"
bareObj['type']="DIGITAL_QUAD_BARE_MODULE"
bareObj['institution']="GL"
bareObj['properties'] = {'SENSOR_TYPE' : '0', 'FECHIP_VERSION': '0'}
bareObj

{'project': 'P',
 'subproject': 'PG',
 'institution': 'GL',
 'componentType': 'BARE_MODULE',
 'type': 'DIGITAL_QUAD_BARE_MODULE',
 'properties': {'SENSOR_TYPE': '0', 'FECHIP_VERSION': '0'}}

In [10]:
### register component
new_bare = myClient.post('registerComponent', json=bareObj)
newCode=new_bare['component']['code']

In [11]:
### get component
compCode=newCode
compObj = myClient.get('getComponent',json={'component':compCode})

In [12]:
# what tests are appropriate?
compTypeCode=compObj['componentType']['code']
compTestList=list(myClient.get('listTestTypes',json={'project':projCode, 'componentType':compTypeCode}))
compTestMap=[{"name":w['name'], "code":w['code']} for w in compTestList]
df_compTest=pd.DataFrame(compTestMap)
df_compTest.drop_duplicates()

,name,code
0,Bare Module Metrology,BARE_MODULE_METROLOGY
1,Mass Measurement,MASS_MEASUREMENT
2,Optical Inspection,OPTICAL_INSPECTION
3,Vender Information,VENDER_INFORMATION


In [13]:
# test format
testCode="BARE_MODULE_METROLOGY"
testObj=myClient.get('generateTestTypeDtoSample',json={'project':projCode, 'code': testCode, 'componentType':compTypeCode, 'requiredOnly' : False})
testObj

{'component': '...',
 'testType': 'BARE_MODULE_METROLOGY',
 'institution': '...',
 'runNumber': '...',
 'date': '2020-12-10T15:37:06.407Z',
 'passed': True,
 'problems': False,
 'results': {'SILICON_X/Y/Z': [5.3346822253279464, 5.3346822253279464],
  'FLEX_Z': [180.59837703686426, 180.59837703686426],
  'SILICON_PLANARITY_STD_DEV': 288.8615285589762,
  'FLEX_PLANARITY_STD_DEV': 0.8772735055366876,
  'FLEX_PLANARITY': 315.3864731284275,
  'SILICON_PLANARITY': 176.31494275829104,
  'FLEX_X/Y': [285.03530221441025, 285.03530221441025]}}

In [14]:
### delete component
myClient.post('deleteComponent', json={'component':newCode})

checkObj=myClient.get('getComponent',json={'component':newCode})
checkObj['state']

'requestedToDelete'